In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.models import Sequential
from keras.layers import Dense
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


['test.csv', 'noaa_weather_qclcd_documentation.pdf', 'train.csv', 'west_nile', 'mapdata_copyright_openstreetmap_contributors.rds', 'sampleSubmission.csv', 'weather.csv', 'mapdata_copyright_openstreetmap_contributors.txt', 'spray.csv']


In [2]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
sample = pd.read_csv('../input/sampleSubmission.csv')
weather = pd.read_csv('../input/weather.csv')

In [3]:
train.head()

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0


In [4]:
test.head()

,Id,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy
0,1,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
1,2,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
2,3,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
3,4,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX SALINARIUS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
4,5,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX TERRITANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9


In [5]:
weather.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,0448,1849,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,-,-,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,0447,1850,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,-,-,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,0446,1851,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9


In [6]:
labels = train.WnvPresent.values

# Not using codesum for this benchmark
weather = weather.drop('CodeSum', axis=1)

# Split station 1 and 2 and join horizontally
weather_stn1 = weather[weather['Station']==1]
weather_stn2 = weather[weather['Station']==2]
weather_stn1 = weather_stn1.drop('Station', axis=1)
weather_stn2 = weather_stn2.drop('Station', axis=1)
weather = weather_stn1.merge(weather_stn2, on='Date')

In [7]:
weather = weather.replace('M', -1)
weather = weather.replace('-', -1)
weather = weather.replace('T', -1)
weather = weather.replace(' T', -1)
weather = weather.replace('  T', -1)

In [8]:
weather.head()

,Date,Tmax_x,Tmin_x,Tavg_x,Depart_x,DewPoint_x,WetBulb_x,Heat_x,Cool_x,Sunrise_x,Sunset_x,Depth_x,Water1_x,SnowFall_x,PrecipTotal_x,StnPressure_x,SeaLevel_x,ResultSpeed_x,ResultDir_x,AvgSpeed_x,Tmax_y,Tmin_y,Tavg_y,Depart_y,DewPoint_y,WetBulb_y,Heat_y,Cool_y,Sunrise_y,Sunset_y,Depth_y,Water1_y,SnowFall_y,PrecipTotal_y,StnPressure_y,SeaLevel_y,ResultSpeed_y,ResultDir_y,AvgSpeed_y
0,2007-05-01,83,50,67,14,51,56,0,2,0448,1849,0,-1,0.0,0.00,29.10,29.82,1.7,27,9.2,84,52,68,-1,51,57,0,3,-1,-1,-1,-1,-1,0.00,29.18,29.82,2.7,25,9.6
1,2007-05-02,59,42,51,-3,42,47,14,0,0447,1850,0,-1,0.0,0.00,29.38,30.09,13.0,4,13.4,60,43,52,-1,42,47,13,0,-1,-1,-1,-1,-1,0.00,29.44,30.08,13.3,2,13.4
2,2007-05-03,66,46,56,2,40,48,9,0,0446,1851,0,-1,0.0,0.00,29.39,30.12,11.7,7,11.9,67,48,58,-1,40,50,7,0,-1,-1,-1,-1,-1,0.00,29.46,30.12,12.9,6,13.2
3,2007-05-04,66,49,58,4,41,50,7,0,0444,1852,0,-1,0.0,-1,29.31,30.05,10.4,8,10.8,78,51,-1,-1,42,50,-1,-1,-1,-1,-1,-1,-1,0.00,29.36,30.04,10.1,7,10.4
4,2007-05-05,66,53,60,5,38,49,5,0,0443,1853,0,-1,0.0,-1,29.40,30.10,11.7,7,12.0,66,54,60,-1,39,50,5,0,-1,-1,-1,-1,-1,-1,29.46,30.09,11.2,7,11.5


In [9]:
def create_month(x):
    return x.split('-')[1]

def create_day(x):
    return x.split('-')[2]

train['month'] = train.Date.apply(create_month)
train['day'] = train.Date.apply(create_day)
test['month'] = test.Date.apply(create_month)
test['day'] = test.Date.apply(create_day)

# Add integer latitude/longitude columns
train['Lat_int'] = train.Latitude.apply(int)
train['Long_int'] = train.Longitude.apply(int)
test['Lat_int'] = test.Latitude.apply(int)
test['Long_int'] = test.Longitude.apply(int)

In [10]:
train.head()

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,month,day,Lat_int,Long_int
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0,05,29,41,-87
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0,05,29,41,-87
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0,05,29,41,-87
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0,05,29,41,-87
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0,05,29,41,-87


In [11]:
train = train.drop(['Address', 'AddressNumberAndStreet','WnvPresent', 'NumMosquitos'], axis = 1)
test = test.drop(['Id', 'Address', 'AddressNumberAndStreet'], axis = 1)


In [12]:
train = train.merge(weather, on='Date')
test = test.merge(weather, on='Date')
train = train.drop(['Date'], axis = 1)
test = test.drop(['Date'], axis = 1)

In [13]:
from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()

from sklearn.preprocessing import scale
colu=['Species','Street','Trap']

for col in colu:
    train[col] = le.fit_transform(train[col])

In [ ]:
train.head()

In [14]:
train = train.ix[:,(train != -1).any(axis=0)]
test = test.ix[:,(test != -1).any(axis=0)]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [15]:
train.head()

,Species,Block,Street,Trap,Latitude,Longitude,AddressAccuracy,month,day,Lat_int,Long_int,Tmax_x,Tmin_x,Tavg_x,Depart_x,DewPoint_x,WetBulb_x,Heat_x,Cool_x,Sunrise_x,Sunset_x,Depth_x,SnowFall_x,PrecipTotal_x,StnPressure_x,SeaLevel_x,ResultSpeed_x,ResultDir_x,AvgSpeed_x,Tmax_y,Tmin_y,Tavg_y,DewPoint_y,WetBulb_y,Heat_y,Cool_y,PrecipTotal_y,StnPressure_y,SeaLevel_y,ResultSpeed_y,ResultDir_y,AvgSpeed_y
0,2,41,32,1,41.954690,-87.800991,9,05,29,41,-87,88,60,74,10,58,65,0,9,0421,1917,0,0.0,0.00,29.39,30.11,5.8,18,6.5,88,65,77,59,66,0,12,0.00,29.44,30.09,5.8,16,7.4
1,3,41,32,1,41.954690,-87.800991,9,05,29,41,-87,88,60,74,10,58,65,0,9,0421,1917,0,0.0,0.00,29.39,30.11,5.8,18,6.5,88,65,77,59,66,0,12,0.00,29.44,30.09,5.8,16,7.4
2,3,62,27,6,41.994991,-87.769279,9,05,29,41,-87,88,60,74,10,58,65,0,9,0421,1917,0,0.0,0.00,29.39,30.11,5.8,18,6.5,88,65,77,59,66,0,12,0.00,29.44,30.09,5.8,16,7.4
3,2,79,109,13,41.974089,-87.824812,8,05,29,41,-87,88,60,74,10,58,65,0,9,0421,1917,0,0.0,0.00,29.39,30.11,5.8,18,6.5,88,65,77,59,66,0,12,0.00,29.44,30.09,5.8,16,7.4
4,3,79,109,13,41.974089,-87.824812,8,05,29,41,-87,88,60,74,10,58,65,0,9,0421,1917,0,0.0,0.00,29.39,30.11,5.8,18,6.5,88,65,77,59,66,0,12,0.00,29.44,30.09,5.8,16,7.4


In [16]:
colu=['month', 'day', 'Tavg_x', 'Depart_x',
       'WetBulb_x', 'Heat_x', 'Cool_x', 'Sunrise_x', 'Sunset_x', 'Depth_x',
       'Tavg_y', 'WetBulb_y', 'Heat_y', 'Cool_y']
col1=['SnowFall_x','PrecipTotal_x', 'StnPressure_x', 'SeaLevel_x', 'AvgSpeed_x','PrecipTotal_y', 'StnPressure_y', 'SeaLevel_y','AvgSpeed_y']

for col in colu:
    train[col]=train[col].astype(str).astype(int)
for col in col1:
    train[col]=train[col].astype(str).astype(float)

In [43]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( train, labels, test_size=0.33, random_state=42)

In [41]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test) 

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


In [34]:
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
log=LogisticRegression(penalty='l2',C=.00001)
log.fit(X_train,y_train)
y_pred = log.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
accuracy
score=f1_score(y_test,y_pred,average='weighted')
score,accuracy

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(0.921965071486473, 0.9475050475915777)

In [35]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=10,oob_score=True ,random_state =42, min_samples_split=25)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test) 
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
accuracy
from sklearn.metrics import f1_score
score=f1_score(y_test,y_pred,average='weighted')
score,accuracy


/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:463: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


(0.9229285977231609, 0.9472166137871358)

In [36]:
from sklearn.svm import SVC
clf = SVC(random_state = 100, kernel='rbf')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
accuracy
from sklearn.metrics import f1_score
score=f1_score(y_test,y_pred,average='weighted')
score,accuracy

/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(0.921965071486473, 0.9475050475915777)

In [37]:
train.head()

,Species,Block,Street,Trap,Latitude,Longitude,AddressAccuracy,month,day,Lat_int,Long_int,Tmax_x,Tmin_x,Tavg_x,Depart_x,DewPoint_x,WetBulb_x,Heat_x,Cool_x,Sunrise_x,Sunset_x,Depth_x,SnowFall_x,PrecipTotal_x,StnPressure_x,SeaLevel_x,ResultSpeed_x,ResultDir_x,AvgSpeed_x,Tmax_y,Tmin_y,Tavg_y,DewPoint_y,WetBulb_y,Heat_y,Cool_y,PrecipTotal_y,StnPressure_y,SeaLevel_y,ResultSpeed_y,ResultDir_y,AvgSpeed_y
0,2,41,32,1,41.954690,-87.800991,9,5,29,41,-87,88,60,74,10,58,65,0,9,421,1917,0,0.0,0.0,29.39,30.11,5.8,18,6.5,88,65,77,59,66,0,12,0.0,29.44,30.09,5.8,16,7.4
1,3,41,32,1,41.954690,-87.800991,9,5,29,41,-87,88,60,74,10,58,65,0,9,421,1917,0,0.0,0.0,29.39,30.11,5.8,18,6.5,88,65,77,59,66,0,12,0.0,29.44,30.09,5.8,16,7.4
2,3,62,27,6,41.994991,-87.769279,9,5,29,41,-87,88,60,74,10,58,65,0,9,421,1917,0,0.0,0.0,29.39,30.11,5.8,18,6.5,88,65,77,59,66,0,12,0.0,29.44,30.09,5.8,16,7.4
3,2,79,109,13,41.974089,-87.824812,8,5,29,41,-87,88,60,74,10,58,65,0,9,421,1917,0,0.0,0.0,29.39,30.11,5.8,18,6.5,88,65,77,59,66,0,12,0.0,29.44,30.09,5.8,16,7.4
4,3,79,109,13,41.974089,-87.824812,8,5,29,41,-87,88,60,74,10,58,65,0,9,421,1917,0,0.0,0.0,29.39,30.11,5.8,18,6.5,88,65,77,59,66,0,12,0.0,29.44,30.09,5.8,16,7.4


In [21]:
import xgboost as xgb
model=xgb.XGBClassifier(random_state=1,learning_rate=0.01)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
score=f1_score(y_test,y_pred,average='weighted')
score
accuracy = accuracy_score(y_test, y_pred)
score,accuracy

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(0.921965071486473, 0.9475050475915777)

In [46]:
model = Sequential()
model.add(Dense(20, input_dim=42, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50, batch_size=10,validation_data=(X_test, y_test))
y_pred = model.predict(X_test)

Train on 7039 samples, validate on 3467 samples
Epoch 1/50
7039/7039 [==============================] - 1s 209us/step - loss: 0.8449 - acc: 0.9476 - val_loss: 0.8461 - val_acc: 0.9475
Epoch 2/50
7039/7039 [==============================] - 1s 152us/step - loss: 0.8449 - acc: 0.9476 - val_loss: 0.8461 - val_acc: 0.9475
Epoch 3/50
7039/7039 [==============================] - 1s 148us/step - loss: 0.8449 - acc: 0.9476 - val_loss: 0.8461 - val_acc: 0.9475
Epoch 4/50
7039/7039 [==============================] - 1s 153us/step - loss: 0.8449 - acc: 0.9476 - val_loss: 0.8461 - val_acc: 0.9475
Epoch 5/50
7039/7039 [==============================] - 1s 150us/step - loss: 0.8449 - acc: 0.9476 - val_loss: 0.8461 - val_acc: 0.9475
Epoch 6/50
7039/7039 [==============================] - 1s 147us/step - loss: 0.8449 - acc: 0.9476 - val_loss: 0.8461 - val_acc: 0.9475
Epoch 7/50
7039/7039 [==============================] - 1s 151us/step - loss: 0.8449 - acc: 0.9476 - val_loss: 0.8461 - val_acc: 0.9475


NameError: name 'X' is not defined

In [49]:
scores = model.evaluate(X_test, y_test)
scores

3467/3467 [==============================] - 0s 33us/step


[0.846118686528767, 0.9475050475743857]

In [51]:
labels=pd.DataFrame(labels)

In [59]:
labels.columns=['ind']
labels.head()

,ind
0,0
1,0
2,0
3,0
4,0


In [63]:
labels.ind.value_counts()

0    9955
1     551
Name: ind, dtype: int64

In [65]:
train.head()

,Species,Block,Street,Trap,Latitude,Longitude,AddressAccuracy,month,day,Lat_int,Long_int,Tmax_x,Tmin_x,Tavg_x,Depart_x,DewPoint_x,WetBulb_x,Heat_x,Cool_x,Sunrise_x,Sunset_x,Depth_x,SnowFall_x,PrecipTotal_x,StnPressure_x,SeaLevel_x,ResultSpeed_x,ResultDir_x,AvgSpeed_x,Tmax_y,Tmin_y,Tavg_y,DewPoint_y,WetBulb_y,Heat_y,Cool_y,PrecipTotal_y,StnPressure_y,SeaLevel_y,ResultSpeed_y,ResultDir_y,AvgSpeed_y
0,2,41,32,1,41.954690,-87.800991,9,5,29,41,-87,88,60,74,10,58,65,0,9,421,1917,0,0.0,0.0,29.39,30.11,5.8,18,6.5,88,65,77,59,66,0,12,0.0,29.44,30.09,5.8,16,7.4
1,3,41,32,1,41.954690,-87.800991,9,5,29,41,-87,88,60,74,10,58,65,0,9,421,1917,0,0.0,0.0,29.39,30.11,5.8,18,6.5,88,65,77,59,66,0,12,0.0,29.44,30.09,5.8,16,7.4
2,3,62,27,6,41.994991,-87.769279,9,5,29,41,-87,88,60,74,10,58,65,0,9,421,1917,0,0.0,0.0,29.39,30.11,5.8,18,6.5,88,65,77,59,66,0,12,0.0,29.44,30.09,5.8,16,7.4
3,2,79,109,13,41.974089,-87.824812,8,5,29,41,-87,88,60,74,10,58,65,0,9,421,1917,0,0.0,0.0,29.39,30.11,5.8,18,6.5,88,65,77,59,66,0,12,0.0,29.44,30.09,5.8,16,7.4
4,3,79,109,13,41.974089,-87.824812,8,5,29,41,-87,88,60,74,10,58,65,0,9,421,1917,0,0.0,0.0,29.39,30.11,5.8,18,6.5,88,65,77,59,66,0,12,0.0,29.44,30.09,5.8,16,7.4


## SMOTE

In [140]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( train, labels, test_size=0.33, random_state=42)

In [141]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(ratio='minority')
X_train, y_train = smote.fit_sample(X_train, y_train)


/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [142]:
X_train=pd.DataFrame(X_train)

In [143]:
X_test.columns

Index(['Species', 'Block', 'Street', 'Trap', 'Latitude', 'Longitude',
       'AddressAccuracy', 'month', 'day', 'Lat_int', 'Long_int', 'Tmax_x',
       'Tmin_x', 'Tavg_x', 'Depart_x', 'DewPoint_x', 'WetBulb_x', 'Heat_x',
       'Cool_x', 'Sunrise_x', 'Sunset_x', 'Depth_x', 'SnowFall_x',
       'PrecipTotal_x', 'StnPressure_x', 'SeaLevel_x', 'ResultSpeed_x',
       'ResultDir_x', 'AvgSpeed_x', 'Tmax_y', 'Tmin_y', 'Tavg_y', 'DewPoint_y',
       'WetBulb_y', 'Heat_y', 'Cool_y', 'PrecipTotal_y', 'StnPressure_y',
       'SeaLevel_y', 'ResultSpeed_y', 'ResultDir_y', 'AvgSpeed_y'],
      dtype='object')

In [144]:
X_train.columns=['Species', 'Block', 'Street', 'Trap', 'Latitude', 'Longitude',
       'AddressAccuracy', 'month', 'day', 'Lat_int', 'Long_int', 'Tmax_x',
       'Tmin_x', 'Tavg_x', 'Depart_x', 'DewPoint_x', 'WetBulb_x', 'Heat_x',
       'Cool_x', 'Sunrise_x', 'Sunset_x', 'Depth_x', 'SnowFall_x',
       'PrecipTotal_x', 'StnPressure_x', 'SeaLevel_x', 'ResultSpeed_x',
       'ResultDir_x', 'AvgSpeed_x', 'Tmax_y', 'Tmin_y', 'Tavg_y', 'DewPoint_y',
       'WetBulb_y', 'Heat_y', 'Cool_y', 'PrecipTotal_y', 'StnPressure_y',
       'SeaLevel_y', 'ResultSpeed_y', 'ResultDir_y', 'AvgSpeed_y']

In [145]:
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
log=LogisticRegression(penalty='l2',C=.00001)
log.fit(X_train,y_train)
y_pred = log.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
accuracy
score=f1_score(y_test,y_pred,average='weighted')
score,accuracy

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(0.7221130330798993, 0.618402076723392)

In [146]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=20,oob_score=True ,random_state =42, min_samples_split=25)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test) 
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
accuracy
from sklearn.metrics import f1_score
score=f1_score(y_test,y_pred,average='weighted')
score,accuracy


/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:463: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


(0.9247192707075893, 0.9267378136717623)

In [147]:
import xgboost as xgb
model=xgb.XGBClassifier(random_state=1,learning_rate=0.01)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
score=f1_score(y_test,y_pred,average='weighted')
score
accuracy = accuracy_score(y_test, y_pred)
score,accuracy

(0.7975840157160625, 0.7202192096913759)

In [148]:
from sklearn.svm import SVC
clf = SVC(random_state = 100, kernel='rbf')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
accuracy
from sklearn.metrics import f1_score
score=f1_score(y_test,y_pred,average='weighted')
score,accuracy

/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


(0.9148108350628869, 0.9077011825785982)

In [149]:
model = Sequential()
model.add(Dense(20, input_dim=42, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50, batch_size=10,validation_data=(X_test, y_test))
y_pred = model.predict(X_test)
scores = model.evaluate(X_test, y_test)
scores

Train on 13340 samples, validate on 3467 samples
Epoch 1/50
13340/13340 [==============================] - 3s 237us/step - loss: 8.0590 - acc: 0.5000 - val_loss: 0.8461 - val_acc: 0.9475
Epoch 2/50
13340/13340 [==============================] - 2s 149us/step - loss: 8.0590 - acc: 0.5000 - val_loss: 0.8461 - val_acc: 0.9475
Epoch 3/50
13340/13340 [==============================] - 2s 151us/step - loss: 8.0590 - acc: 0.5000 - val_loss: 0.8461 - val_acc: 0.9475
Epoch 4/50
13340/13340 [==============================] - 2s 153us/step - loss: 8.0590 - acc: 0.5000 - val_loss: 0.8461 - val_acc: 0.9475
Epoch 5/50
13340/13340 [==============================] - 2s 150us/step - loss: 8.0590 - acc: 0.5000 - val_loss: 0.8461 - val_acc: 0.9475
Epoch 6/50
13340/13340 [==============================] - 2s 145us/step - loss: 8.0590 - acc: 0.5000 - val_loss: 0.8461 - val_acc: 0.9475
Epoch 7/50
13340/13340 [==============================] - 2s 149us/step - loss: 8.0590 - acc: 0.5000 - val_loss: 0.8461 - v

[0.846118686528767, 0.9475050475743857]